# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# Ignoring hidden files like .ipynb_checkpoints present in the /event_data folder

for root, dirs, files in os.walk(filepath):
    if ('.ipynb_checkpoints' in root):
        continue    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)


/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Query1 : select artist_name,song ,song_length from artists where sessionid=338 and iteminsession=4;
# Table Name : artists
# col1: artist_name
# col2: song
# col3: song_length
# col4: sessionid
# col5: iteminsession
# PRIMARY KEY : sessionid,iteminsession
# Based on the query, sessionid and iteminsession combinely make the record unique. One sessionid has many iteminsessions so both sessionid and \
# iteminsession define unique records in table
query = "CREATE TABLE IF NOT EXISTS artists "
query = query + "(artist_name text, song text, song_length float, sessionid int,iteminsession int, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [12]:
# Insert data into artists table from event_datafile_new.csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artists(artist_name,song,song_length,sessionid,iteminsession)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]),int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name,song ,song_length from artists where sessionid=338 and iteminsession=4"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['artist_name','song','song_length'])
    print(result)
except Exception as e:
    print(e)


  artist_name                             song  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Query  2 : select artist,song,firstname,lastname from music where userid = 10 and sessionid = 182
# Table name : music
#col1: artist_name
#col2: song
#col3: userid
#col4: firstname
#col5: lastname
#col6: sessionid
#col7: iteminsession
#PRIMARY KEY : ((userid,sessionid),iteminsession)
# Based on the query statement,userid and sessionid combinely defines the unique record in the table and as per data retrival\
# on sorted iteminsession, the clustering column would be iteminsession
query = "CREATE TABLE IF NOT EXISTS music "
query = query + "(artist_name text, song text,userid int, firstname text,lastname text, sessionid int,iteminsession int, PRIMARY KEY ((userid,sessionid),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [15]:
## Insert data into music table from event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music(artist_name,song,userid,firstname,lastname,sessionid,iteminsession)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9],int(line[10]), line[1], line[4],int(line[8]),int(line[3])))

In [25]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist_name,song,firstname,lastname from music where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['artist_name','song','firstname','lastname'])
    print(result)
except Exception as e:
    print(e)


         artist_name                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Query 3: select firstname,lastname from users where song='All Hands Against His Own'
#Table name: users
#col1: userid
#col2: firstname
#col3: lastname
#col4: gender
#col5: song
# PRIMARY KEY : song,userid
# In this table,song and userid combinely define unique record in the table, as alone song can not be unique per record.
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(userid int, firstname text,lastname text,gender text,song text, PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)   
                   

In [18]:
## Insert data into users table from event_datafile_new.csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users(userid,firstname,lastname,gender,song)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1],line[4], line[2], line[9]))

In [23]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select firstname,lastname from users where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    result = pd.DataFrame(rows._current_rows,columns=['Firstname','Lastname'])
    print(result)
except Exception as e:
    print(e)
  

    Firstname Lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [24]:
## TO-DO: Drop the table before closing out the sessions
query_list = ["DROP TABLE artists","DROP TABLE music ","DROP TABLE users"]
for query in query_list:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()